# Climate Change Sentiment Analysis

#### Sebastian Vargas, Baxter Romero, Johnny Wilcox, Trajan Pei, Caleb Bettcher


## EDA:

Hydrate Data Set $\newline$
Drop any missing long/lattitude rows $\newline$
Create global subset of tweets (COL: Timestamp, Tweet, Sentiment Score)$\newline$
Create subsets of global subset for individual countries$\newline$
Create and evaluate subsets of different countries

Questions we should try to answer with EDA:

Describe details about the dataset you’ve chosen – $\newline$
How many examples(size after analysis), $\newline$
What statistics do you observe about the examples (Both columns, and data shape)$\newline$
How did you create training/validation/test sets? $\newline$
